In [1]:
import scrapy
import json
from scrapy.crawler import CrawlerProcess
from scrapy.spiders import CrawlSpider
from scrapy.utils.project import get_project_settings

In [2]:
from scrapy import optional_features
optional_features.remove('boto')

In [3]:
import string

In [4]:
companies = []
with open('companies.csv','r') as f:
    for line in f:
        companies.append(line[:-1])



In [5]:
len(companies)

36

In [6]:
companies = map(lambda x:x.lower(),companies)
companies_l = map(lambda x:x.split(),companies)

In [7]:
title = ''
urls = ''
post_text = []
max_pages = 2
paras = []
class YsItem(scrapy.Item):
    # define the fields for your item here like:
    cname = scrapy.Field()
    title = scrapy.Field()
    post = scrapy.Field()
    
class YsSpider(CrawlSpider):
    global max_pages
    name = "ysposts"
    allowed_domains = ["yourstory.com"]
    start_urls = []
    for c in companies[0:36]:
        for page in range(1,20):
            start_urls.append('http://yourstory.com/?s='+c+'&paged='+str(page))
    
    def process_title(self,t):
        exclude = set(string.punctuation)
        t = ''.join(ch if ch not in exclude else ' ' for ch in list(t))
        t = t.strip()
        t = t.lower()
        t = t.split()
        return t
    
    def get_cname(self,title):
        cname = []
        for word in title:
            for comp in companies_l:
                if word in comp:
                    cname.append(comp)
        return cname
        
    
    def getURLS(self,title):
        url_vec = []
        for t in title:
            t = self.process_title(t)
            uv = 0
            for word in t:
                for comp in companies_l:
                    if word in comp:
                        uv=1
                        #print(t)
            url_vec.append(uv)
        return url_vec

    def parse(self, response):
            global title
            global urls
            title = response.xpath('''//div[@class="title-small 
                                    bentonCondensed bold 
                                    color-black-2 truncate-2"] 
                                   /text()''').extract()
            title = [x.strip().encode('ascii','replace') for x in title]
            if len(title[0])!=0:                
                urls = response.xpath('//a[@class="block"]/@href')
                url_vec = self.getURLS(title)
                for i in range(len(url_vec)):
                    if url_vec[i]==1:
                        url = urls[i].extract()
                        yield scrapy.Request(url, callback=self.parse_dir_contents)
 


    def parse_dir_contents(self, response):
        global paras
        global post_text
        item = YsItem()
        t = response.xpath('//h3[@class="title color-ys"]/text()').extract()
        #print(t[0])
        it = self.process_title(t[0].encode('ascii','replace'))
        #print(it)
        item['title'] = it
        item['cname'] = self.get_cname(it)
        paras = response.xpath('//div[@class="ys_post_content text"]//p')
        post_text = ' '
        for para in paras:
            post_text += " ".join(para.xpath('text()').extract())
            post_text += " "
        post_text = self.process_title(post_text.encode('ascii','replace'))
        item['post'] = post_text
        if len(item['cname'])!=0:
            with open('ysnote_new.json','a') as f:
                json.dump({'cname':item.get('cname'),
                           'title':item.get('title'),
                           'post':item.get('post')},f)
        



In [8]:
process = CrawlerProcess(get_project_settings())
process.crawl(YsSpider)
process.start()

INFO:scrapy.utils.log:Scrapy 1.0.5 started (bot: scrapybot)
2016-03-25 01:24:33 [scrapy] INFO: Scrapy 1.0.5 started (bot: scrapybot)
INFO:scrapy.utils.log:Optional features available: ssl, http11
2016-03-25 01:24:33 [scrapy] INFO: Optional features available: ssl, http11
INFO:scrapy.utils.log:Overridden settings: {}
2016-03-25 01:24:33 [scrapy] INFO: Overridden settings: {}
INFO:scrapy.middleware:Enabled extensions: CloseSpider, TelnetConsole, LogStats, CoreStats, SpiderState
2016-03-25 01:24:33 [scrapy] INFO: Enabled extensions: CloseSpider, TelnetConsole, LogStats, CoreStats, SpiderState
INFO:scrapy.middleware:Enabled downloader middlewares: HttpAuthMiddleware, DownloadTimeoutMiddleware, UserAgentMiddleware, RetryMiddleware, DefaultHeadersMiddleware, MetaRefreshMiddleware, HttpCompressionMiddleware, RedirectMiddleware, CookiesMiddleware, ChunkedTransferMiddleware, DownloaderStats
2016-03-25 01:24:33 [scrapy] INFO: Enabled downloader middlewares: HttpAuthMiddleware, DownloadTimeoutMid